In [1]:
import pickle
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

print("데이터 로딩 중...")
file_path = "mon_standard.pkl"
with open(file_path, 'rb') as fi:
    data = pickle.load(fi)

데이터 로딩 중...


In [2]:
# 데이터 전처리
X_cul_size = []
y = []
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE

    for sample in data[i]:
        cul_size_seq = []
        cumulative_size = 0
        for index, c in enumerate(sample):
            dr = 1 if c > 0 else -1
            cumulative_size += dr * 512
            cul_size_seq.append(cumulative_size)        
        X_cul_size.append(cul_size_seq)
        y.append(label)

# 각 배열의 길이를 최대 길이에 맞춰 늘리고 부족한 부분은 0으로 채웁니다.
max_length = max(len(arr) for arr in X_cul_size)

def pad_sequence(sequence, max_length):
    return sequence + [0] * (max_length - len(sequence))

X_cul_size_padded = [pad_sequence(arr, max_length) for arr in X_cul_size]

X_cul_size_array = np.array(X_cul_size_padded)
X_train_cul_size, X_test_cul_size, y_train_cul_size, y_test_cul_size = train_test_split(X_cul_size_array, y, test_size=0.2, random_state=42)

# StandardScaler를 사용하여 X_cul_size 데이터 정규화
scaler = StandardScaler()
X_train_cul_size_scaled = scaler.fit_transform(X_train_cul_size)
X_test_cul_size_scaled = scaler.transform(X_test_cul_size)

In [4]:
# SVM 모델 학습 및 평가
svm_model_cul_size = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_cul_size.fit(X_train_cul_size_scaled, y_train_cul_size)

y_pred_svm_cul_size = svm_model_cul_size.predict(X_test_cul_size_scaled)
accuracy_svm_cul_size = accuracy_score(y_test_cul_size, y_pred_svm_cul_size)

print(f"svm 모델 정확도 (X_cul_size): {accuracy_svm_cul_size * 100:.2f}%")

svm 모델 정확도 (X_cul_size): 84.03%


In [5]:
# rf
rf_model_cul_size = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_cul_size.fit(X_train_cul_size_scaled, y_train_cul_size)

y_pred_rf_cul_size = rf_model_cul_size.predict(X_test_cul_size_scaled)
accuracy_rf_cul_size = accuracy_score(y_test_cul_size, y_pred_rf_cul_size)
print(f"랜덤 포레스트 모델 정확도 (cul_size): {accuracy_rf_cul_size * 100:.2f}%")

랜덤 포레스트 모델 정확도 (cul_size): 91.82%


In [6]:
# KNN 
knn_model_cul_size = KNeighborsClassifier(n_neighbors=5)
knn_model_cul_size.fit(X_train_cul_size_scaled, y_train_cul_size)

y_pred_knn_cul_size = knn_model_cul_size.predict(X_test_cul_size_scaled)

accuracy_knn_cul_size = accuracy_score(y_test_cul_size, y_pred_knn_cul_size)
print(f"KNN 모델 정확도 (X_cul_size): {accuracy_knn_cul_size * 100:.2f}%")

KNN 모델 정확도 (X_cul_size): 69.08%
